In [44]:
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date
from scraper import BoligScraper
import numpy as np
import geopy
pd.options.display.max_columns = 500 
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
## To-do:
# 1. Automate address-to-coordinate process
# 2. Scrape tinglysing w. address information
# 3. Manage new estates for sale, and estates disappearing from boligsiden.dk
# 4. Figure out how to plot geo-coordinates on a map?

 # Get Coordinate of Address w. GeoPy

In [59]:
# Get random address
df = pd.read_pickle('./data/boligsiden_2020-07-31.pkl')
sample = df.sample(n=1)

# Print address
print('PLACE')
print(list(sample.address)[0])
print(list(sample.postal)[0])
print(list(sample.city)[0])

# Print coordinates
print('\nCOORDINATES')
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Egevangshusene 47, Taastrup, Denmark')
print(f'Latitude: {location.latitude}')
print(f'Longitude: {location.longitude}')

PLACE
Poul Buås Vej 34, 2. th
9000
Aalborg

COORDINATES
Latitude: 55.646261
Longitude: 12.283713


# Scrape Boligsiden.dk

In [2]:
scraper = BoligScraper()
df = scraper.scrape()

Scraping..
Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Concatenating DataFrames..
Saving scraped data to disk..
Scraping finished!
Cleaning numeric columns..
Done!


# Clean DataFrame

In [17]:
def clean_cols(df):

    # Convert numeric columns to float
    cols = ['paymentCash', 'downPayment', 'paymentExpenses', 'paymentGross','paymentNet', 'areaResidential', 
            'numberOfRooms', 'areaParcel', 'salesPeriod', 'areaPaymentCash', 'areaWeighted', 'salesPeriodTotal']

    def if_dash(x):
        if not x.isnumeric():
            x = 0
        return x

    for col in cols:
        df[col] = df[col].apply(lambda x: x.replace('.', ''))
        df[col] = df[col].apply(lambda x: if_dash(x))
        df[col] = df[col].astype(float)
    
    return df


# Quick Groupby's

## Most Valuable Estate Company

In [ ]:
mvp = df.groupby('agentChainName')['paymentCash'].sum().sort_values(ascending=False)
mvp[:10]

## Most Expensive Cities

In [ ]:
mec = df.groupby('city')['paymentCash'].sum().sort_values(ascending=False)
mec[:10]

## Kortest Liggetid

In [ ]:
kl = df.groupby('city')['salesPeriodTotal'].mean().sort_values()#ascending=False)
kl[10:60]